## first lab: CLEANING NUMERICAL DATA

##### importing libraries

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

##### displaying main features (head, shape, info)

In [11]:
customer_df = pd.read_csv("we_fn_use_c_marketing_customer_value_analysis.csv")
display(customer_df.head())
display(customer_df.shape)
display(customer_df.info)

FileNotFoundError: [Errno 2] No such file or directory: 'we_fn_use_c_marketing_customer_value_analysis.csv'

##### snake case label and formatting datetime column

In [ ]:
cols=[]
for i in customer_df.columns:
    cols.append(i.lower())
customer_df.columns = cols
customer_df.columns = customer_df.columns.str.replace(' ', '_')
customer_df = customer_df.rename(columns = {"employmentstatus":"employment_status"})
customer_df.columns

setting max columns bcs it looks better and there are only 24 columns, it wouldn't hurt, I guess

In [ ]:
pd.set_option('display.max_columns', None)

turning column effective_to_date into datetime type

In [ ]:
customer_df["effective_to_date"] = customer_df["effective_to_date"].astype("datetime64[ns]")
type(customer_df["effective_to_date"][0])

##### separating continuous and discrete columns

first: separate all numerical columns from the dataframe

In [ ]:
customer_df.head(1)

In [ ]:
numerical_df = customer_df.select_dtypes(include=np.number)

then see which column(s) contains how many unique values.

we want to put columns with up to 200s unique values to our discrete_df and the rest to continuous_df

In [ ]:
numerical_df.nunique()

this is me reviewing python fundamentals before making the function

In [ ]:
# learning how to get the count of unique values
# learning how to turn something to a list
# (numerical_df.nunique().tolist())

In [ ]:
# Esraa sent me this. forgot to ask what this line was for
# for col in data.columns:
#     print (len(data[col].value_counts()),'        ',col)

In [ ]:
# learning how to create a new dataframe
# a = pd.DataFrame()
# a

In [ ]:
# learning pandas indexing
# print(numerical_df.iloc[:,0])
# numerical_df.columns[0]

In [ ]:
# from Erin
# for i in data_num.columns:
#     if len(data_num[i].unique()) > 300:
#         continuous_lst.append(i)
#     else:
#         discrete_lst.append(i)

making function to split numerical_df into discrete and cont, based on the count of unique values of each column

In [ ]:
def con_dis(df):
    con = pd.DataFrame()
    dis = pd.DataFrame()
    for i in range(len(df.nunique().tolist())):
        if df.nunique().tolist()[i] > 250:
            con[df.columns[i]] = df.iloc[:,i]
        else:
            dis[df.columns[i]] = df.iloc[:,i]
    return con, dis

continuous_df, discrete_df = con_dis(numerical_df)
display(discrete_df.nunique())
display(continuous_df.nunique())
display(discrete_df.head())
display(continuous_df.head())

##### heatmap and correlation matrix

In [ ]:
plt.figure(figsize=(12, 9))
correlations_matrix = customer_df.corr()
sns.heatmap(correlations_matrix, annot=True)
plt.show()

correlation matrix

In [ ]:
correlations_matrix

In [ ]:
# theres some correlations, one of the highest would be between total_claim_amount and monthly_premium_auto
# if we are about to build models, we might build it without dropping those correlated columns

In [ ]:
customer_df.columns

##### plotting the columns

In [ ]:
for i in discrete_df.columns:
    sns.displot(x = i, data = discrete_df, bins=30)
    plt.show()

In [ ]:
# as for income, we could make bins
# for number_of_open_complaints we can leave it as it is
# for number_of_policies, we might want to group 4-9 into one category

In [ ]:
for i in continuous_df.columns:
    sns.histplot(x = i, data = continuous_df, bins=30)
    plt.show()

In [ ]:
# comment what you see in the plots
# will do it later

##### looking for outliers: use boxplots and/or others

In [ ]:
for i in discrete_df.columns:
    sns.histplot(x = i, data = discrete_df)
    plt.show()

In [ ]:
for i in continuous_df.columns:
    sns.boxplot(x = i, data = continuous_df)
    plt.show()

In [ ]:
# boxplot seems better for the continuous_df, while
# it doesnt bring us much information with the discrete_df
# for discrete_df we find histplot a better option to see the outliers

for the outliers: we're probably gonna drop them. but it depends on the variable and the values inside them. we can either use mean, mode, or drop them 

In [ ]:
customer_df.isna().sum()

no NaNs. nice

## next lab: CLEANING CATEGORICAL DATA 

##### defining categorical_df

In [ ]:
categorical_df = customer_df.select_dtypes(include=[np.object])
categorical_df.head(2)

checking for NaNs

In [ ]:
categorical_df.isna().sum()

no NaNs. nice

##### check all unique values in all columns, to decide what to do with each of them

In [ ]:
for i in categorical_df.columns:
    print("column", i+":")
    display(categorical_df[i].value_counts(dropna = False))

some of the columns might need to be changed into binaries (?), like
instead of F and M in gender, we could change it into 1 for female and 0 for male

we might use onehot to encode almost all the columns here except customer. either exclude it so it doesnt get onehot,
or just drop the whole column customer, since the values are all-unique. we won't (probably) get any significant informations from an all-unique column

we could try seeing the correlation matrix to check whether the subcategories (L1 - L3) from the column policy have
collinearity with other columns. if they show muilticollinearity, we could drop the column policy and keep just the 
policy type instead. (will do the corr_matrix later/tomorrow/some other time)

for the policy/policy-type, either one needs to be dropped, since it just repeats the information. most probably the column policy has some multicollinearity with other columns so we might want to drop it

as for renew_offer_type, it doesnt hurt to keep it. unless we find multicollinearity in heatmap

continuing our "what we could do"

- combine the value master and doctor as a "higher educational degree"
- combine college with bachelor as "college" or some other name

- make 1 bigger category consists of luxury cars, luxury suv, and sport cars

- combine medical leave with employed. or search the definition about it. but combining them sounds nice
- might combine medical_leave and disabled into 1 new category
- might combine retired with unemployed

##### executing the plan(s)

dropping the whole column "customer"

In [ ]:
try:
    categorical_df = categorical_df.drop(["customer"], axis = 1)
    print("deleted the column.")
except:
    print("column already deleted.")
categorical_df.head(0)

dropping the whole column "policy"

In [ ]:
try:
    categorical_df = categorical_df.drop(["policy"], axis = 1)
    print("deleted the column.")
except:
    print("column already deleted.")
categorical_df.head(0)

redefining unique values we want to keep in column "vehicle_class"

In [ ]:
categorical_df['vehicle_class'] = np.where(categorical_df.vehicle_class.isin(["Sports Car","Luxury SUV","Luxury Car"]),'Luxury Cars', categorical_df.vehicle_class)
categorical_df['vehicle_class'].value_counts()

redefining unique values we want to keep in column "education"

In [ ]:
def clean_education(x):
    if x in ["Master", "Doctor"]:
        return 'higher educational degree'
    elif x in ["Bachelor", "College"]:
        return "baccalaureate degree"
    else:
        return x

categorical_df['education'] = categorical_df['education'].apply(clean_education)
categorical_df['education'].value_counts()


redefining unique values we want to keep in column "employment_status"

In [ ]:
categorical_df['employment_status'] = np.where(categorical_df.employment_status.isin(["Medical Leave","Disabled"]),'M-D_related_leave', categorical_df.employment_status)
categorical_df['employment_status'] = np.where(categorical_df.employment_status.isin(["Retired"]),'Unemployed', categorical_df.employment_status)
categorical_df['employment_status'].value_counts()


final check

In [ ]:
categorical_df.head(2)

just listing what's done here
- dropped the column "customer"
- dropped column "policy"
- simplified "education" column
- simplified "vehicle_class" column
- simplified "employment_status" column


## next lab: FEATURE EXTRACTION 

##### plotting something

In [ ]:
for i in categorical_df.columns:
    sns.histplot(data = categorical_df[i])
    plt.show()

##### plotting datetime

just checking what the datetime column name was

In [ ]:
customer_df.head(1)

In [ ]:
plt.figure(figsize=(15,4))
sns.lineplot(data=customer_df, x="effective_to_date", y='total_claim_amount',markers=True)

##### simao

In [ ]:
# see simao's plotting. it looks coool

## next lab: COMPARING REGRESSION MODEL

##### removing outliers

In [ ]:
for i in continuous_df.columns:
    sns.boxplot(x = i, data = continuous_df)
    plt.show()

In [ ]:
def drop_upper_outliers(df, column, outlier):
    df = df[df[column] < outlier]
    return df

In [ ]:
c_customer_df = customer_df.copy()
c_customer_df = drop_upper_outliers(c_customer_df, "total_claim_amount", 2200)
c_customer_df = drop_upper_outliers(c_customer_df, "customer_lifetime_value", 54000)
c_customer_df = drop_upper_outliers(c_customer_df, "monthly_premium_auto", 210)
c_customer_df.shape

In [ ]:
for i in discrete_df.columns:
    sns.histplot(x = i, data = discrete_df)
    plt.show()

Im so confused here, do I need to redefine the categoricals again? since the number of rows has decreased

##### redefining categoricals, numericals, discrete, continuous (again?)

In [ ]:
def con_dis(df):
    con = pd.DataFrame()
    dis = pd.DataFrame()
    for i in range(len(df.nunique().tolist())):
        if df.nunique().tolist()[i] > 250:
            con[df.columns[i]] = df.iloc[:,i]
        else:
            dis[df.columns[i]] = df.iloc[:,i]
    return con, dis

categorical_df = c_customer_df.select_dtypes(include=[np.object])
numerical_df = c_customer_df.select_dtypes(include=np.number)
continuous_df, discrete_df = con_dis(numerical_df)

redo all the "cleaning categorical" for the new df (after removing outliers)

In [ ]:
# drop column "customer"
try:
    categorical_df = categorical_df.drop(["customer"], axis = 1)
    print("deleted the column.")
except:
    print("column already deleted.")
categorical_df.head(0)

# drop column "policy"
try:
    categorical_df = categorical_df.drop(["policy"], axis = 1)
    print("deleted the column.")
except:
    print("column already deleted.")
categorical_df.head(0)

# reducing unique values from "vehicle_class"
categorical_df['vehicle_class'] = np.where(categorical_df.vehicle_class.isin(["Sports Car","Luxury SUV","Luxury Car"]),'Luxury Cars', categorical_df.vehicle_class)

# reducing unique values from "education"
def clean_education(x):
    if x in ["Master", "Doctor"]:
        return 'higher educational degree'
    elif x in ["Bachelor", "College"]:
        return "baccalaureate degree"
    else:
        return x

categorical_df['education'] = categorical_df['education'].apply(clean_education)

# reducing unique values in "employment_status"
categorical_df['employment_status'] = np.where(categorical_df.employment_status.isin(["Medical Leave","Disabled"]),'M-D_related_leave', categorical_df.employment_status)
categorical_df['employment_status'] = np.where(categorical_df.employment_status.isin(["Retired"]),'Unemployed', categorical_df.employment_status)


##### concat discrete and continuous as numerical; concat numerical and categorical as data

i decided to standardize both discrete and continuous because if I dont, the continuous will have a way smaller value range, while discrete still has a wider value range

In [ ]:
numerical_new = pd.concat([discrete_df, continuous_df], axis = 1)

In [ ]:
data = pd.concat([numerical_new, categorical_df], axis = 1)

In [ ]:
data.columns

In [ ]:
# get data
# clean data, dropping any rows and columns that need to be dropped (1)
# select features (could postpone this)
# X/y split
# Train/Test split
# split both Train and Test in numericals and categoricals
# transformations on numericals:
#     fit ONLY on numericals_train
#     transform BOTH numericals_train and numericals_test
# encoding categoricals
#     fit ONLY on categricals_train
#     encode BOTH categoricals_train and categoricals_test
# combine numericals_train and categoricals_train into train_processed
# combine numericals_test and categoricals_test into test_processed
# define model
# fit model on train_processed
# evaluate (score) model on test_processed + sanity check (does it make sense?)
# save model and transformers/encoders (2)

##### X-y-split, then train-test-split

In [ ]:
y = data[["total_claim_amount"]].copy()
X = data.drop(columns = "total_claim_amount", axis = 1).copy()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

##### splitting each train and test set to numericals and categoricals, then do scaling and/or encoding

In [ ]:
numericals_train = X_train.select_dtypes(np.number)
numericals_test = X_test.select_dtypes(np.number)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
transformer = MinMaxScaler().fit(numericals_train)
numericals_train_standardized = transformer.transform(numericals_train)
numericals_test_standardized = transformer.transform(numericals_test)
numericals_train_standardized.shape

In [ ]:
categoricals_train = X_train.select_dtypes(object)
categoricals_test = X_test.select_dtypes(object)
display(categoricals_train.shape)
display(categoricals_train.head(0))

In [ ]:
ordinal_cols = ["coverage","employment_status","location_code","vehicle_size"]
onehot_cols = ["state","marital_status","policy_type","renew_offer_type","sales_channel","vehicle_class"]
cat_ordinal_train = pd.DataFrame(categoricals_train, columns = ordinal_cols)
cat_ordinal_test = pd.DataFrame(categoricals_test, columns = ordinal_cols)
cat_onehot_train = pd.DataFrame(categoricals_train, columns = onehot_cols)
cat_onehot_test = pd.DataFrame(categoricals_test, columns = onehot_cols)


In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown = 'error', drop = 'first').fit(cat_onehot_train)
onehot_train_encoded = encoder.transform(cat_onehot_train).toarray()
onehot_test_encoded = encoder.transform(cat_onehot_test).toarray()

from sklearn.preprocessing import OrdinalEncoder
ord_encoder = OrdinalEncoder(handle_unknown= "error").fit(cat_ordinal_train)
ordinal_train_encoded = ord_encoder.transform(cat_ordinal_train)
ordinal_test_encoded = ord_encoder.transform(cat_ordinal_test)

##### final step: concat everything to processed X_train and X_test

In [ ]:
categoricals_train_encoded = np.concatenate((onehot_train_encoded, ordinal_train_encoded), axis = 1)
categoricals_test_encoded = np.concatenate((onehot_test_encoded, ordinal_test_encoded), axis = 1)

In [ ]:
X_train_processed = np.concatenate((numericals_train_standardized,categoricals_train_encoded),axis=1)
X_test_processed = np.concatenate((numericals_test_standardized,categoricals_test_encoded),axis=1)

##### importing libraries and defining function

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression

In [ ]:
def make_models_and_get_scores(xtrain, xtest, ytrain, ytest):
    model = []
    train = []
    test = []
    test_result = pd.DataFrame()

    KNN = KNeighborsRegressor(n_neighbors=5, p = 1)
    KNN.fit(xtrain, ytrain)
    model.append("K-Neighbors")
    KNN_R2_score_train = KNN.score(xtrain, ytrain)
    train.append(KNN_R2_score_train)
    KNN_R2_score_test = KNN.score(xtest, ytest)
    test.append(KNN_R2_score_test)
    # KNN_predictions_train = KNN.predict(xtrain)
    # KNN_predictions_test = KNN.predict(xtest)

    lm = linear_model.LinearRegression()
    lm.fit(xtrain,ytrain)
    model.append("Linear Regression")
    LM_predictions_train = lm.predict(xtrain)
    LM_R2_score_train = r2_score(ytrain, LM_predictions_train)
    train.append(LM_R2_score_train)
    LM_predictions_test = lm.predict(xtest)
    LM_R2_score_test = r2_score(ytest, LM_predictions_test)
    test.append(LM_R2_score_test)

    regr = MLPRegressor(random_state = 1, max_iter = 700).fit(xtrain, ytrain)
    model.append("MLP Regressor")
    mlp_score_train = regr.score(xtrain, ytrain)
    train.append(mlp_score_train)
    mlp_score_test = regr.score(xtest, ytest)
    test.append(mlp_score_test)
    # mlp_predictions_train = regr.predict(xtrain)
    # mlp_predictions_test = regr.predict(xtest)

    test_result["model"] = model
    test_result["r2_train"] = train
    test_result["r2_test"] = test

    return test_result


##### run the function to get test results in dataframe format

In [ ]:
scores = make_models_and_get_scores(X_train_processed, X_test_processed, y_train, y_test)
scores

In [ ]:
# discussing the result
# of the 3 models, MLP regressor seem to show the best scores both in train and test
# linreg scores poorly :(
# KNN scores almost as good as MLP, but MLP works a lot slower as the max_iter gets higher
# MLP scores are, up to certain point, related to the number of max_iter
# after max_iter = 700, the test score doesnt get much higher anymore than shown above
# but with max_iter = 700 the MLP calculation takes more time than KNN
# so if we're too lazy to wait, or if we have bigger dataframe and want to experiment iteratively,
# MLP might not be too desirable as it takes a lot of time, and even more if you're looping